<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON611
### HW_4 -  SQL
#### Due: Nov 18

We will be using the sample databases that come from the DBeaver program (use the Database Installation guide to assist you on installing it). We will be using the sample tables that are provided there. Save your work as a SQL script with answers to specific questions commented out properly labled.

## 1. Music and Album Data
---
We will be using the following tables from DBeaver:
• Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that has the columns Album Title, Artist, Track Name, Genre, Composer, Length of Song (minutes) 
---

### Q A (5pts) My answer: 
---
My logic: beacsue there does not exists `right join` or `full outer join` in `SQLite` version, so for this question and all the qs I used `left-join` <br>

The first question is just `left join` and select all the cols needed <br>
<br>

- but I did convert the `t.Milseconds` to `float` data type to make the output in minutes more accurate instead of just one integer


```sql

SELECT T.NAME                      AS "Track Name",
       T.composer,
       T.milliseconds * 1. / 60000 AS "length of music",
       A.title                     AS "Album Title",
       AR.NAME                     AS "Artist Name",
       GEN.NAME                    AS "Genre "
FROM   track AS T
       LEFT JOIN album A
              ON A.albumid = T.albumid
       LEFT JOIN genre GEN
              ON T.genreid = GEN.genreid
       LEFT JOIN artist AR
              ON A.artistid = AR.artistid
              
              
```

#### B. (5 points). Output a frequency table for how many letters are in an album title and a song title. (i.e. 3 columns: number of letters, number of album titles with that amount of letters, number of song titles with that amount of letters). What is the average and range of amount of letters?
---

###  Q B (5pts)  My Answer: 

---
Output a frequency table for how many letters are in an album title and a song title. (i.e. 3 columns: number of letters, number of album titles with that amount of letters, number of song titles with that amount of letters). <br>

My logic: 
I used `subquery` for this question <br> 
- first, within the subquery, I count letters in the songs/Track.Name 
- then, in another subquery, I count letters in the Album.Title
- I join the previous 2 subqueries using the `left join`
- at the end, I display the 3 cols that this question requested;
---
variable name demostration <br>
- `numofchrs` $\rightarrow$  intersection part of the letter count <br>
- `song_name_count` $\rightarrow$ the letter count of the letter in the Track/song.Name <br>
- `album_name_count` $\rightarrow$ the letter count of the album title <br>
since there is no `full outer join` and `right join`, I will just take their intersection part, hence I used `join` instead of `left join`

### Q B.1
--- 
Output a frequency table for how many letters are in an album title and a song title. (i.e. 3 columns: number of letters, number of album titles with that amount of letters, number of song titles with that amount of letters


<br>
<br>

```sql 
SELECT t1.numofchrs,
       t1.song_name_count,
       t2.album_name_count
FROM   (SELECT Length(T.NAME) AS 'NumOfChrs',
               Count(*)       AS song_name_count
        FROM   track T
        GROUP  BY numofchrs
        ORDER  BY 1 DESC) t1
       JOIN (SELECT Length(AL.title) AS 'NumOfChrsAL',
                    Count(*)         AS album_name_count
             FROM   album AL
             GROUP  BY numofchrsal) t2
         ON t1.numofchrs = t2.numofchrsal
ORDER  BY 1 DESC


```

Q B.2 
---

What is the average and range of amount of letters?

<br>
<br>


```sql
SELECT Avg(( T.milliseconds * 1. ) / 60000),
       ( Max(T.milliseconds) - Min(T.milliseconds) ) * 1. / 60000 AS Rng
FROM   track T
       LEFT JOIN genre Gen
              ON T.genreid = Gen.genreid
WHERE  Gen.NAME = 'Heavy Metal'



```





#### C. (10 points). How many songs did each composer write? How many artists composed their own music? Output all of the Artists who had songs that were longer than 2.5 minutes and they did not write their own songs.
---

### Q C .1 My Anwers:
---
how many songs did each composer write 
<br>
<br>
```sql

SELECT   Count(*) AS 'Num of songs' ,
         composer
FROM     track
WHERE    composer notnull
GROUP BY composer
ORDER BY "Num of songs" DESC


```

### Q C .2 
--- 
How many artists composed their own music 
<br>
<br>
- becasue sometimes, artists apear to be one of the composers and sometimes, composer appears to belong to one of the artists ; <br>
- so by using the `INSTR` function which count if the artist is in the composer col and vice versa (if the composer in the artist col. And the result is stored in the col `if_contains`.


```sql
SELECT if_contains,
       Count(if_contains)
FROM   (SELECT T.composer,
               Art.NAME,
               CASE
                 WHEN ( Instr(T.composer, Art.NAME) >= 1 )
                       OR ( Instr(Art.NAME, T.composer) >= 1 )THEN 1
                 ELSE 0
               END AS 'If_contains'
        FROM   track T
               LEFT JOIN album Al
                      ON T.albumid = Al.albumid
               LEFT JOIN artist Art
                      ON Al.artistid = Art.artistid)
GROUP  BY if_contains

```

### Q C.3 
---
Output all of the Artists who had songs that were longer than 2.5 minutes and they did not write their own songs.
<br>
<br>

```sql 
SELECT Count(*)
FROM   track T
       LEFT JOIN album Al
              ON T.albumid = Al.albumid
       LEFT JOIN artist Art
              ON Al.artistid = Art.artistid
WHERE  ( ( Instr(T.composer, Art.NAME) < 1 )
          OR Instr(Art.NAME, T.composer) < 1 )
       AND T.milliseconds > 150000




```





#### D. (10 points). How many albums contain songs that only have an even number of seconds in the length of the song and the songs are all MPEG audio files?
---

### Q D My Answers: 
---
- I use a subquery to find all the `album.ablumID` that does not satisfied the requirement
- then I use a main query to filter out the `albumid` that not in the query list which subquery provided


```sql

SELECT al.albumid
FROM   album al
WHERE  al.albumid NOT IN (SELECT alb.albumid
                          FROM   album alb
                                 LEFT JOIN track t
                                        ON al.albumid = t.albumid
                          WHERE  ( t.milliseconds / 1000 ) % 2 != 0
                                  OR t.mediatypeid != 1)



```


#### E. (10 points). How large, in terms of Megabytes are each album? On average, how large, in terms of bytes, is a pop song? What is the range and average duration of a heavy metal song in minutes?
---


### E.1  My  Answers
---
How large, in terms of Megabytes are each album

```sql
SELECT t.albumid, 
       t.bytes / 1048576           AS MB, 
       Sum(t.bytes * 1. / 1048576) AS MB_size 
FROM   track t 
       LEFT JOIN album al 
              ON t.albumid = al.albumid 
GROUP  BY al.albumid 

```

### E.2 
---
On average, how large, in terms of bytes, is a pop song
<br> 


```sql
SELECT Avg(T.bytes)
FROM   track T
       LEFT JOIN genre Gen
              ON T.genreid = Gen.genreid
WHERE  Gen.NAME LIKE ( 'Pop' )




```


### E.3 
What is the range and average duration of a heavy metal song in minutes?

---

```sql 

SELECT Avg(( T.milliseconds * 1. ) / 60000)                       AS Minute_, 
       ( Max(T.milliseconds) - Min(T.milliseconds) ) * 1. / 60000 AS Rng 
FROM   track T 
       LEFT JOIN genre Gen 
              ON T.genreid = Gen.genreid 
WHERE  Gen.NAME = 'Heavy Metal' 



```



## 2. Customer Data
---
We will be using the following tables from DBeaver:
• Customer • Invoice • InvoiceLine • Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that shows the number of customers in each city, and also have shows the state and country that the city is located in.
---

### Q A  My Answer:
---
Output a table that shows the number of customers in each city
<br>
```sql

SELECT Count(*),
       state,
       country
FROM   customer
WHERE  state IS NOT NULL
GROUP  BY city,
          state,
          country

```

#### B. (10 points). How much in total was spent on music per year? What is the average amount a person spent on music in a year? What is the average amount a person with a gmail account spent on music in a year?
---

### B.1
--- 
 How much in total was spent on music per year? 
 
 
 ```sql
 SELECT Strftime('%Y', i.invoicedate) AS year_index, 
       Sum(i.total) 
FROM   invoice i 
GROUP  BY year_index 

 ```

### B.2
--- 
What is the average amount a person spent on music in a year

```sql
SELECT t1.year_index, 
       t1.total_spending / t1.total_cust AS year_avg_spending 
FROM   (SELECT Strftime('%Y', i.invoicedate) AS year_index, 
               Sum(i.total)                  AS total_spending, 
               Count(DISTINCT customerid)    AS total_cust 
        FROM   invoice i 
        GROUP  BY year_index) t1 
GROUP  BY year_index 
```


### B.3  
---
What is the average amount a person with a gmail account spent on music in a year

```sql
SELECT t1.year_index, 
       t1.total_spending / t1.total_cust AS year_avg_spending 
FROM   (SELECT Strftime('%Y', i.invoicedate) AS year_index, 
               Sum(i.total)                  AS total_spending, 
               Count(DISTINCT i.customerid)  AS total_cust 
        FROM   invoice i 
               LEFT JOIN customer cust 
                      ON cust.customerid = I.customerid 
        WHERE  cust.email LIKE ( '%gmail%' ) 
        GROUP  BY year_index) t1 
GROUP  BY year_index 
```

#### C. (10 points). What year-month combination had the most spending on music? Which year-month combi- nation had the most money spent on pop music?
---

### C.1 
--- 
What year-month combination had the most spending on music?

```sql
SELECT Strftime('%Y-%m', i.invoicedate) AS year_index, 
       Sum(i.total)                     AS spend_on_music 
FROM   invoice i 
       LEFT JOIN customer cust 
              ON cust.customerid = I.customerid 
GROUP  BY year_index 
ORDER  BY spend_on_music DESC 
```


### C.2
---
Which year-month combi- nation had the most money spent on `pop` music?
```sql
SELECT Strftime('%Y-%m', i.invoicedate) AS year_mth_index, 
       gen.NAME, 
       i.invoiceid, 
       Sum(i.total)                     AS pop_sum 
FROM   invoiceline AS IL 
       LEFT JOIN invoice AS I 
              ON IL.invoiceid = I.invoiceid 
       LEFT JOIN track T 
              ON T.trackid = IL.trackid 
       LEFT JOIN genre Gen 
              ON Gen.genreid = T.genreid 
WHERE  Gen.NAME = 'Pop' 
GROUP  BY year_mth_index 
ORDER  BY pop_sum DESC
    
       
```



#### D. (10 points). What artists has the most songs bought that are in the pop genre? What is the total money spent on these songs? Which customer is spending the most on pop songs?
---

### D 
---
What artists has the most songs bought that are in the pop genre?
<br>

```sql

SELECT arts.NAME,
       Count(*)     AS amt_songs,
       Sum(I.total) AS total_spending,
       cust.firstname,
       cust.lastname
FROM   invoiceline AS IL
       LEFT JOIN invoice AS I
              ON IL.invoiceid = I.invoiceid
       LEFT JOIN customer AS cust
              ON cust.customerid = I.customerid
       LEFT JOIN track T
              ON T.trackid = IL.trackid
       LEFT JOIN genre Gen
              ON Gen.genreid = T.albumid
       LEFT JOIN album Al
              ON Al.albumid = T.albumid
       LEFT JOIN artist arts
              ON arts.artistid = AL.artistid
WHERE  Gen.NAME = 'Pop'
GROUP  BY arts.NAME
ORDER BY Total_Spending DESC
```



#### E. (10 points). Output how many songs each artist is selling in each country? Who is selling the post outside of the US? On average, how many songs is each artist selling outside of the US?
---

### E.1
---
<br>
Output how many songs each artist is selling in each country
```sql

SELECT Count(DISTINCT t.trackid) AS counter, 
       t.trackid, 
       arts.NAME, 
       cust.country 
FROM   invoiceline AS IL 
       LEFT JOIN invoice AS I 
              ON IL.invoiceid = I.invoiceid 
       LEFT JOIN customer AS cust 
              ON cust.customerid = I.customerid 
       LEFT JOIN track T 
              ON T.trackid = IL.trackid 
       LEFT JOIN genre Gen 
              ON Gen.genreid = T.albumid 
       LEFT JOIN album Al 
              ON Al.albumid = T.albumid 
       LEFT JOIN artist arts 
              ON arts.artistid = AL.artistid 
GROUP  BY arts.artistid, 
          cust.country  
```

### E.2
---
Who is selling the most outside of the US?
<br>


```sql
SELECT Sum(t1.track_counter) AS sum_tracks, 
       t1.NAME 
FROM   (SELECT Count(DISTINCT t.trackid) AS Track_counter, 
               t.trackid, 
               arts.NAME, 
               cust.country 
        FROM   invoiceline AS IL 
               LEFT JOIN invoice AS I 
                      ON IL.invoiceid = I.invoiceid 
               LEFT JOIN customer AS cust 
                      ON cust.customerid = I.customerid 
               LEFT JOIN track T 
                      ON T.trackid = IL.trackid 
               LEFT JOIN genre Gen 
                      ON Gen.genreid = T.albumid 
               LEFT JOIN album Al 
                      ON Al.albumid = T.albumid 
               LEFT JOIN artist arts 
                      ON arts.artistid = AL.artistid 
        GROUP  BY arts.artistid, 
                  cust.country) t1 
GROUP  BY t1.NAME 
ORDER BY sum_tracks DESC

```

### E.3
---
 On average, how many songs is each artist selling outside of the US


```sql
SELECT Sum(t2.sum_tracks) * 1. / Count(DISTINCT t2.NAME) * 1. 
FROM   (SELECT Sum(t1.track_counter) AS sum_tracks, 
               t1.NAME 
        FROM   (SELECT Count(DISTINCT t.trackid) AS Track_counter, 
                       t.trackid, 
                       arts.NAME, 
                       cust.country 
                FROM   invoiceline AS IL 
                       LEFT JOIN invoice AS I 
                              ON IL.invoiceid = I.invoiceid 
                       LEFT JOIN customer AS cust 
                              ON cust.customerid = I.customerid 
                       LEFT JOIN track T 
                              ON T.trackid = IL.trackid 
                       LEFT JOIN genre Gen 
                              ON Gen.genreid = T.albumid 
                       LEFT JOIN album Al 
                              ON Al.albumid = T.albumid 
                       LEFT JOIN artist arts 
                              ON arts.artistid = AL.artistid 
                GROUP  BY arts.artistid, 
                          cust.country) t1 
        GROUP  BY t1.NAME 
        ORDER  BY sum_tracks DESC) t2 

          

          
```


## 3. Window Functions (20 points)
---

#### A. (10 points). Return a list of Customers that assigns a sequential integer to each customer and resets the number when the country of the customer changes for Customers are not from Canada and Brazil and whose email address have `yahoo` or `gmail'
---

### A.1
---

<br>
assign row number to different custmers 


```sql
SELECT cust.customerid, 
       Row_number() 
         OVER ( 
           ORDER BY cust.customerid) 
FROM   customer cust 
```


### A.2
---
<br>
reset the row number when both condition is met / filter out cols; 
```sql
SELECT cust.customerid, 
       cust.country, 
       cust.email,
       Row_number() 
         OVER ( 
           partition BY cust.Country 
         ) AS reset_col 
FROM   customer cust
WHERE cust.country NOT IN ('Canada', 'Brazil') AND 
         (cust.email LIKE ('%gmail%') OR cust.email LIKE ('%yahoo%'))
```


#### B. (10 points). Compute the rank for each track in each album based on the track’s length, then append all the tracks that have a lengthrank of 2 and 4 
---

### B 
---

```sql
SELECT *
FROM   (SELECT NAME,
               milliseconds,
               albumid,
               Dense_rank()
                 OVER (
                   partition BY albumid
                   ORDER BY milliseconds) lenRank
        FROM   track)
WHERE  lenrank = 2
        OR lenrank = 4

```

## BONUS (20 points)
---
- Using SQLITE syntax [make your own SQLITE table](https://www.sqlitetutorial.net/sqlite-create-table/) and set of data. You will need to create a table of 5 columns and 10 rows containing any information you find interesting. (5 points)
- You should create/import your table need to [dbeaver](https://docs.roguewave.com/en/zendstudio/current/content/create_manage_database_with_dbeaver.htm). (5 points)
- Perform 3 different calculations using your data in order to tell a story of your data set. (10 points)

** FEEL FREE TO USE/EXPLORE OTHER SOURCES**


1. create my own table
---

- create a $10 \times 5$ table

```sql
CREATE TABLE leagueoflegends_champion 
  ( 
     cham_id       INTEGER PRIMARY KEY, 
     champion_name TEXT NOT NULL, 
     win_rate      FLOAT NOT NULL, 
     game_played   INTEGER NOT NULL, 
     kda           TEXT NOT NULL 
  ) 
  
  

```



2. create or import table need to `dbeaver` 

3. perform 3 different calculation using your data in order to tell a story of the dataset